In [1]:
# for cleaning up after tests
import os
dir = "data/test_dir/"
for f in os.listdir(dir):
    if ".bin" in f:
        os.remove(os.path.join(dir, f))

In [2]:
from banners import *
import csv_relations
import os
# get all dirs with data and order by date
data_dirs = []
root = "data/"
for dd in os.listdir(root):
    d = os.path.join(root, dd)
    print(d)
    if os.path.isdir(d):
        dir_files = Dir(d, dd)
        for f in os.listdir(d):
            if ".bag" in f:
                dir_files.bag = os.path.join(d, f)
            if ".csv" in f:
                dir_files.clicks = os.path.join(d, f)
        data_dirs.append(dir_files)

print(data_dirs)

TOPICS = ['/current_pose', '/cam0/camera/image_raw', '/cam1/camera/image_raw', '/therm/image_raw_throttle']

processed_data = []

# process each bag
for a in data_dirs:
    cc = CameraCombo(a)
    bag = rosbag.Bag(a.bag)
    for topic, msg, t in bag.read_messages(TOPICS):
        # print(topic)
        # stack 3 images together
        if topic == '/therm/image_raw_throttle':
            # print("thermal")
            cc.flir = msg
        if topic == '/cam1/camera/image_raw':
            # print("rgb")
            cc.rgb = msg
        if topic == '/cam0/camera/image_raw':
            # print("noir")
            cc.noir = msg
        if topic == "/current_pose":
            print("stacking")
            cc.pose = msg.pose.pose
            cc.stack(t)
    processed_data.append(cc)
    cc.done()


    # get pose of stack
    # compute coords(xmin, ymin & xmax, ymax) of each stack


data/test_dir
[Dir(path='data/test_dir', date='test_dir', bag='data/test_dir/2023-01-08-10-27-04.bag', clicks='data/test_dir/Data0.csv')]
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stackin

In [3]:
# processed_data[0].parsed
data_dirs[0].clicks

'data/test_dir/Data0.csv'

In [4]:
import csv
import csv_relations
proc_click = []

# go over the CSVs
for a in data_dirs:
    C = csv_relations.CSVAugmented(a.path)
    with open(a.clicks) as clicks:
        reader = csv.reader(clicks)
        for line in reader:
            # breakdown line
            pt = [float(line[0]), float(line[1])]
            health = int(line[-1])
            
            C.jungler(pt, health, processed_data)

    C.done()
    proc_click.append(C)